In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import math
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
# read multiple csv from data folder
files = []
for file in os.listdir('data'):
    if file.endswith('.csv'):
        files.append(file)
print(files)

In [ ]:
df = pd.read_csv('data/' + files[0])
df

In [ ]:
df_new = df.groupby("time").agg({"ssr":"mean", "u100":"mean", "v100":"mean"}).reset_index()

In [ ]:
df_new

In [ ]:

df_daily = df_hourly.resample('D', on="timestamps")["photo","wind_on_offshore"].mean().reset_index()



In [ ]:
df_new["windspeed"] = np.sqrt(df_new['u100']**2 + df_new['v100']**2)
df_new

In [ ]:
df_new["timestamps"] = pd.to_datetime(df_new["time"])

In [ ]:
df_new_daily = df_new.resample('D', on="timestamps")["ssr","windspeed"].mean().reset_index()
df_new_daily

In [ ]:
# plot them
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_new_daily["timestamps"], y=df_new_daily["ssr"], mode='lines', name='ssr'))
fig.add_trace(go.Scatter(x=df_new_daily["timestamps"], y=df_new_daily["windspeed"], mode='lines', name='windspeed'))
fig.show()


In [ ]:
df_realized_supply = pd.read_csv('data/' + files[2], sep=';')
df_realized_supply.columns

In [ ]:
df_realized_supply

In [ ]:
df_realized_supply = df_realized_supply[['Date from', 'Date to', "Photovoltaic [MW]", "Wind Offshore [MW] ", "Wind Onshore [MW]"]]
df_realized_supply.columns

In [ ]:
df_copy = df_realized_supply.copy()

In [ ]:
# df_copy["Wind Onshore [MW]"] = df_copy["Wind Onshore [MW]"].apply(lambda x: x.replace(',', '.'))
df_copy["wind_off"] = df_copy["Wind Offshore [MW] "].str.replace('.', '').str.replace(",",".").astype(float)
df_copy["wind_on"] = df_copy["Wind Onshore [MW]"].str.replace('.', '').str.replace(",",".").astype(float)
df_copy["photo"] = df_copy["Photovoltaic [MW]"].str.replace('.', '').str.replace(",",".").astype(float)





In [ ]:
df_copy

In [ ]:
df_realized_supply

In [ ]:
df_copy["wind_on_offshore"] = df_copy["wind_off"] + df_copy["wind_on"]
df_copy


In [ ]:
df_copy = df_copy.drop(columns=["Wind Offshore [MW] ", "Wind Onshore [MW]"])
df_copy = df_copy.drop(columns=["Photovoltaic [MW]"])

In [ ]:
df_copy

In [ ]:
df_realized_supply = df_copy

In [ ]:
df_realized_supply["timestamps"] = pd.to_datetime(df_realized_supply["Date from"])
df_realized_supply['month_year'] = df_realized_supply['timestamps'].dt.strftime('%Y-%m')
df_realized_supply['day'] = df_realized_supply['timestamps'].dt.strftime('%d')
df_realized_supply["fullhour"] = df_realized_supply['timestamps'].dt.strftime('%H:%M')

In [ ]:
df_hourly = df_realized_supply.resample('H', on="timestamps")["photo","wind_on_offshore"].mean().reset_index()

df_daily = df_hourly.resample('D', on="timestamps")["photo","wind_on_offshore"].mean().reset_index()



In [ ]:
df_hourly

In [ ]:
df_hourly.sort_values(by="timestamps", inplace=True)
df_agg = df_hourly
fig = go.Figure()
x_axis = df_agg["timestamps"]

fig  = px.line(x=x_axis, y=df_agg.photo,
                    )
# title
fig.update_layout(
    title="Average photovoltaic power supply per hour",
    xaxis_title="hour",
    yaxis_title="Power supply [MW]",
)
fig.show()

In [ ]:
df_hourly.sort_values(by="timestamps", inplace=True)
df_agg = df_hourly
fig = go.Figure()
x_axis = df_agg["timestamps"]

fig  = px.line(x=x_axis, y=df_agg.wind_on_offshore,
                    )
# title
fig.update_layout(
    title="Average Wind power supply per hour",
    xaxis_title="hour",
    yaxis_title="Wind onshore and offshore power supply [MW]",
)
fig.show()

In [ ]:
df_daily.sort_values(by="timestamps", inplace=True)
df_agg = df_daily
fig = go.Figure()
x_axis = df_agg["timestamps"]

fig  = px.line(x=x_axis, y=df_agg.wind_on_offshore,
                    )
# title
fig.update_layout(
    title="Average Wind power supply per hour",
    xaxis_title="hour",
    yaxis_title="Wind onshore and offshore power supply [MW]",
)
fig.show()

In [ ]:
df_daily.sort_values(by="timestamps", inplace=True)
df_agg = df_daily
fig = go.Figure()
x_axis = df_agg["timestamps"]

fig  = px.line(x=x_axis, y=df_agg.photo,
                    )
# title
fig.update_layout(
    title="Average Wind power supply per hour",
    xaxis_title="hour",
    yaxis_title="Wind onshore and offshore power supply [MW]",
)
fig.show()

## Merge 

In [ ]:
df_merged_daily = pd.merge(df_new_daily, df_daily, on="timestamps", how="inner")
df_merged_daily

In [ ]:
df_hourly_merged = pd.merge(df_new, df_hourly, on="timestamps", how="inner")
df_hourly_merged

In [ ]:
import pandas as pd
import numpy as np
import pylab as pl
from pykalman import KalmanFilter

# Load your data into a DataFrame
data = df_merged_daily  # Replace with your actual file path

# Extract the relevant columns
timestamps = data['timestamps']
ssr = data['ssr'].values
photo = data['photo'].values

# Normalize the data
ssr = (ssr - np.mean(ssr)) / np.std(ssr)
photo = (photo - np.mean(photo)) / np.std(photo)

# Prepare matrices
x = photo.reshape(-1, 1)  # Hidden states (photo)
z = ssr.reshape(-1, 1)  # Observations (ssr)

M = len(photo)

# Estimate A, W, H, Q using the closed-form solutions
A = np.dot(x[1:].T, x[:-1]) @ np.linalg.inv(np.dot(x[:-1].T, x[:-1]))
W = (np.dot(x[1:].T, x[1:]) - np.dot(A, np.dot(x[:-1].T, x[1:]))) / (M - 1)
H = np.dot(z.T, x) @ np.linalg.inv(np.dot(x.T, x))
Q = (np.dot(z.T, z) - np.dot(H, np.dot(x.T, z))) / M

# Convert to correct shapes
A = A.reshape(1, 1)
W = W.reshape(1, 1)
H = H.reshape(1, 1)
Q = Q.reshape(1, 1)

# Initialize Kalman Filter with estimated parameters
kf = KalmanFilter(
    transition_matrices=A,
    observation_matrices=H,
    transition_covariance=W,
    observation_covariance=Q,
    initial_state_mean=x[0],
    initial_state_covariance=np.eye(1)
)

# Filter the observations to estimate the states
filtered_state_estimates = kf.filter(z)[0]

# Plot results
pl.figure(figsize=(16, 6))
lines_true = pl.plot(x, linestyle='-', color='b', label='True PV Output (photo)')
lines_obs = pl.plot(z, linestyle=':', color='m', label='SSR (observation)')
lines_filt = pl.plot(filtered_state_estimates, linestyle='--', color='g', label='Filtered PV Output (state estimate)')
pl.legend()
pl.xlabel('Time')
pl.ylabel('Normalized Value')
pl.show()


In [ ]:
import pandas as pd
import numpy as np
import pylab as pl
from pykalman import KalmanFilter
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Load your data into a DataFrame
data = df_merged_daily

# Extract the relevant columns
timestamps = data['timestamps']
ssr = data['ssr'].values
photo = data['photo'].values

# Normalize the data
ssr = (ssr - np.mean(ssr)) / np.std(ssr)
photo = (photo - np.mean(photo)) / np.std(photo)

# Prepare matrices
x = photo.reshape(-1, 1)  # Hidden states (photo)
z = ssr.reshape(-1, 1)  # Observations (ssr)

M = len(photo)

# Estimate A, W, H, Q using the closed-form solutions
A = np.dot(x[1:].T, x[:-1]) @ np.linalg.inv(np.dot(x[:-1].T, x[:-1]))
W = (np.dot(x[1:].T, x[1:]) - np.dot(A, np.dot(x[:-1].T, x[1:]))) / (M - 1)
H = np.dot(z.T, x) @ np.linalg.inv(np.dot(x.T, x))
Q = (np.dot(z.T, z) - np.dot(H, np.dot(x.T, z))) / M

# Convert to correct shapes
A = A.reshape(1, 1)
W = W.reshape(1, 1)
H = H.reshape(1, 1)
Q = Q.reshape(1, 1)

# Initialize Kalman Filter with estimated parameters
kf = KalmanFilter(
    transition_matrices=A,
    observation_matrices=H,
    transition_covariance=W,
    observation_covariance=Q,
    initial_state_mean=x[0],
    initial_state_covariance=np.eye(1)
)

# Filter the observations to estimate the states
filtered_state_estimates = kf.filter(z)[0]

# Calculate evaluation metrics
mae = mean_absolute_error(x, filtered_state_estimates)
mse = mean_squared_error(x, filtered_state_estimates)
rmse = np.sqrt(mse)
r2 = r2_score(x, filtered_state_estimates)

print(f"MAE: {mae:.4f}")
print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R-squared: {r2:.4f}")

# Forecast future values
n_forecasts = 10  # Number of future steps to forecast
last_state = filtered_state_estimates[-1]
forecasts = []

for _ in range(n_forecasts):
    last_state = A @ last_state
    forecasts.append(last_state.item())

# Plot results
pl.figure(figsize=(16, 6))
lines_true = pl.plot(x, linestyle='-', color='b', label='True PV Output (photo)')
# lines_obs = pl.plot(z, linestyle=':', color='m', label='SSR (observation)')
lines_filt = pl.plot(filtered_state_estimates, linestyle='-', color='g', label='Filtered PV Output (state estimate)')
pl.legend()
pl.xlabel('Time')
pl.ylabel('Normalized Value')

# Plot forecasts
future_timestamps = range(len(photo), len(photo) + n_forecasts)
pl.plot(future_timestamps, forecasts, linestyle='-', color='r', label='Forecasted PV Output')
pl.legend()
pl.show()


In [ ]:
import pandas as pd
import numpy as np
import pylab as pl
from pykalman import KalmanFilter
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
df_hourly_merged.fillna(0, inplace=True)
# Load your data into a DataFrame
data = df_hourly_merged

# Extract the relevant columns
timestamps = data['timestamps']
ssr = data['ssr'].values
photo = data['photo'].values

# Normalize the data
ssr = (ssr - np.mean(ssr)) / np.std(ssr)
photo = (photo - np.mean(photo)) / np.std(photo)

# Prepare matrices
x = photo.reshape(-1, 1)  # Hidden states (photo)
z = ssr.reshape(-1, 1)  # Observations (ssr)

M = len(photo)

# Estimate A, W, H, Q using the closed-form solutions
A = np.dot(x[1:].T, x[:-1]) @ np.linalg.inv(np.dot(x[:-1].T, x[:-1]))
W = (np.dot(x[1:].T, x[1:]) - np.dot(A, np.dot(x[:-1].T, x[1:]))) / (M - 1)
H = np.dot(z.T, x) @ np.linalg.inv(np.dot(x.T, x))
Q = (np.dot(z.T, z) - np.dot(H, np.dot(x.T, z))) / M

# Convert to correct shapes
A = A.reshape(1, 1)
W = W.reshape(1, 1)
H = H.reshape(1, 1)
Q = Q.reshape(1, 1)

# Initialize Kalman Filter with estimated parameters
kf = KalmanFilter(
    transition_matrices=A,
    observation_matrices=H,
    transition_covariance=W,
    observation_covariance=Q,
    initial_state_mean=x[0],
    initial_state_covariance=np.eye(1)
)

# Filter the observations to estimate the states
filtered_state_estimates = kf.filter(z)[0]

# Calculate evaluation metrics
mae = mean_absolute_error(x, filtered_state_estimates)
mse = mean_squared_error(x, filtered_state_estimates)
rmse = np.sqrt(mse)
r2 = r2_score(x, filtered_state_estimates)

print(f"MAE: {mae:.4f}")
print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R-squared: {r2:.4f}")

# Forecast future values
n_forecasts = 10  # Number of future steps to forecast
last_state = filtered_state_estimates[-1]
forecasts = []

for _ in range(n_forecasts):
    last_state = A @ last_state
    forecasts.append(last_state.item())

# Plot results
pl.figure(figsize=(16, 6))
lines_true = pl.plot(x, linestyle='-', color='b', label='True PV Output (photo)')
lines_obs = pl.plot(z, linestyle=':', color='m', label='SSR (observation)')
lines_filt = pl.plot(filtered_state_estimates, linestyle='-', color='g', label='Filtered PV Output (state estimate)')
pl.legend()
pl.xlabel('Time')
pl.ylabel('Normalized Value')

# Plot forecasts
future_timestamps = range(len(photo), len(photo) + n_forecasts)
pl.plot(future_timestamps, forecasts, linestyle='-', color='r', label='Forecasted PV Output')
pl.legend()
pl.show()


In [ ]:
import pandas as pd
import numpy as np
import pylab as pl
from pykalman import KalmanFilter
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Load your data into a DataFrame
data = df_merged_daily # Replace with your actual file path

# Extract the relevant columns
timestamps = data['timestamps']
windspeed = data['windspeed'].values
wind_on_offshore = data['wind_on_offshore'].values

# Normalize the data
windspeed = (windspeed - np.mean(windspeed)) / np.std(windspeed)
wind_on_offshore = (wind_on_offshore - np.mean(wind_on_offshore)) / np.std(wind_on_offshore)

# Prepare matrices
x = wind_on_offshore.reshape(-1, 1)  # Hidden states (wind_on_offshore)
z = windspeed.reshape(-1, 1)  # Observations (windspeed)

M = len(wind_on_offshore)

# Estimate A, W, H, Q using the closed-form solutions
A = np.dot(x[1:].T, x[:-1]) @ np.linalg.inv(np.dot(x[:-1].T, x[:-1]))
W = (np.dot(x[1:].T, x[1:]) - np.dot(A, np.dot(x[:-1].T, x[1:]))) / (M - 1)
H = np.dot(z.T, x) @ np.linalg.inv(np.dot(x.T, x))
Q = (np.dot(z.T, z) - np.dot(H, np.dot(x.T, z))) / M

# Convert to correct shapes
A = A.reshape(1, 1)
W = W.reshape(1, 1)
H = H.reshape(1, 1)
Q = Q.reshape(1, 1)

# Initialize Kalman Filter with estimated parameters
kf = KalmanFilter(
    transition_matrices=A,
    observation_matrices=H,
    transition_covariance=W,
    observation_covariance=Q,
    initial_state_mean=x[0],
    initial_state_covariance=np.eye(1)
)

# Filter the observations to estimate the states
filtered_state_estimates = kf.filter(z)[0]

# Calculate evaluation metrics
mae = mean_absolute_error(x, filtered_state_estimates)
mse = mean_squared_error(x, filtered_state_estimates)
rmse = np.sqrt(mse)
r2 = r2_score(x, filtered_state_estimates)

print(f"MAE: {mae:.4f}")
print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R-squared: {r2:.4f}")

# Forecast future values
n_forecasts = 10  # Number of future steps to forecast
last_state = filtered_state_estimates[-1]
forecasts = []

for _ in range(n_forecasts):
    last_state = A @ last_state
    forecasts.append(last_state.item())

# Plot results
pl.figure(figsize=(16, 6))
lines_true = pl.plot(x, linestyle='-', color='b', label='True Wind On/Offshore (wind_on_offshore)')
# lines_obs = pl.plot(z, linestyle=':', color='m', label='Windspeed (observation)')
lines_filt = pl.plot(filtered_state_estimates, linestyle='-', color='g', label='Filtered Wind On/Offshore (state estimate)')
pl.legend()
pl.xlabel('Time')
pl.ylabel('Normalized Value')

# Plot forecasts
future_timestamps = range(len(wind_on_offshore), len(wind_on_offshore) + n_forecasts)
pl.plot(future_timestamps, forecasts, linestyle='-', color='r', label='Forecasted Wind On/Offshore')
pl.legend()
pl.show()


In [ ]:
import pandas as pd
import numpy as np
import pylab as pl
from pykalman import KalmanFilter
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Load your data into a DataFrame
data = df_merged_daily  # Replace with your actual file path

# Extract the relevant columns
timestamps = data['timestamps']
windspeed = data['windspeed'].values
wind_on_offshore = data['wind_on_offshore'].values

# Normalize the data
windspeed = (windspeed - np.mean(windspeed)) / np.std(windspeed)
wind_on_offshore = (wind_on_offshore - np.mean(wind_on_offshore)) / np.std(wind_on_offshore)

# Split the data into training (3/4) and testing (1/4)
train_size = int(len(windspeed) * 0.75)
train_windspeed = windspeed[:train_size]
train_wind_on_offshore = wind_on_offshore[:train_size]
test_windspeed = windspeed[train_size:]
test_wind_on_offshore = wind_on_offshore[train_size:]

# Prepare matrices for training
x_train = train_wind_on_offshore.reshape(-1, 1)  # Hidden states (wind_on_offshore)
z_train = train_windspeed.reshape(-1, 1)  # Observations (windspeed)

M = len(x_train)

# Estimate A, W, H, Q using the closed-form solutions
A = np.dot(x_train[1:].T, x_train[:-1]) @ np.linalg.inv(np.dot(x_train[:-1].T, x_train[:-1]))
W = (np.dot(x_train[1:].T, x_train[1:]) - np.dot(A, np.dot(x_train[:-1].T, x_train[1:]))) / (M - 1)
H = np.dot(z_train.T, x_train) @ np.linalg.inv(np.dot(x_train.T, x_train))
Q = (np.dot(z_train.T, z_train) - np.dot(H, np.dot(x_train.T, z_train))) / M

# Convert to correct shapes
A = A.reshape(1, 1)
W = W.reshape(1, 1)
H = H.reshape(1, 1)
Q = Q.reshape(1, 1)

# Initialize Kalman Filter with estimated parameters
kf = KalmanFilter(
    transition_matrices=A,
    observation_matrices=H,
    transition_covariance=W,
    observation_covariance=Q,
    initial_state_mean=x_train[0],
    initial_state_covariance=np.eye(1)
)

# Perform one-step-ahead predictions on the test set
filtered_state_means = []
predicted_state_means = []

current_state_mean = x_train[-1]
current_state_covariance = np.eye(1)

for t in range(len(test_windspeed)):
    # Predict the next state (one-step-ahead forecast)
    current_state_mean, current_state_covariance = kf.filter_update(
        current_state_mean,
        current_state_covariance,
        observation=test_windspeed[t].reshape(1, 1)
    )
    predicted_state_means.append(current_state_mean[0, 0])
    # Update the filter with the actual observed value
    filtered_state_means.append(test_wind_on_offshore[t])

# Calculate evaluation metrics
mae = mean_absolute_error(filtered_state_means, predicted_state_means)
mse = mean_squared_error(filtered_state_means, predicted_state_means)
rmse = np.sqrt(mse)
r2 = r2_score(filtered_state_means, predicted_state_means)

print(f"MAE: {mae:.4f}")
print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R-squared: {r2:.4f}")

# Plot results
pl.figure(figsize=(16, 6))
pl.plot(range(train_size, train_size + len(test_windspeed)), filtered_state_means, linestyle='-', color='b', label='True Wind On/Offshore (wind_on_offshore)')
pl.plot(range(train_size, train_size + len(test_windspeed)), predicted_state_means, linestyle='--', color='g', label='Predicted Wind On/Offshore')
pl.legend()
pl.xlabel('Time')
pl.ylabel('Normalized Value')
pl.show()


In [ ]:
import pandas as pd
import numpy as np
import pylab as pl
from pykalman import KalmanFilter
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Load your data into a DataFrame
data = df_merged_daily  # Replace with your actual file path

# Extract the relevant columns
timestamps = data['timestamps']
windspeed = data['windspeed'].values
wind_on_offshore = data['wind_on_offshore'].values

# Normalize the data
windspeed = (windspeed - np.mean(windspeed)) / np.std(windspeed)
wind_on_offshore = (wind_on_offshore - np.mean(wind_on_offshore)) / np.std(wind_on_offshore)

# Split the data into training (3/4) and testing (1/4)
train_size = int(len(windspeed) * 0.75)
train_windspeed = windspeed[:train_size]
train_wind_on_offshore = wind_on_offshore[:train_size]
test_windspeed = windspeed[train_size:]
test_wind_on_offshore = wind_on_offshore[train_size:]

# Prepare matrices for training
x_train = train_wind_on_offshore.reshape(-1, 1)  # Hidden states (wind_on_offshore)
z_train = train_windspeed.reshape(-1, 1)  # Observations (windspeed)

M = len(x_train)

# Estimate A, W, H, Q using the closed-form solutions
A = np.dot(x_train[1:].T, x_train[:-1]) @ np.linalg.inv(np.dot(x_train[:-1].T, x_train[:-1]))
W = (np.dot(x_train[1:].T, x_train[1:]) - np.dot(A, np.dot(x_train[:-1].T, x_train[1:]))) / (M - 1)
H = np.dot(z_train.T, x_train) @ np.linalg.inv(np.dot(x_train.T, x_train))
Q = (np.dot(z_train.T, z_train) - np.dot(H, np.dot(x_train.T, z_train))) / M

# Convert to correct shapes
A = A.reshape(1, 1)
W = W.reshape(1, 1)
H = H.reshape(1, 1)
Q = Q.reshape(1, 1)

# Initialize state and covariance
current_state_mean = x_train[-1]
current_state_covariance = np.eye(1)

# Perform one-step-ahead predictions on the test set
filtered_state_means = []
predicted_state_means = []

for t in range(len(test_windspeed)):
    # Extrapolate (Predict)
    # Predict the next state: \hat{\mathbf{x}}_{n+1,n} = \mathbf{F} \hat{\mathbf{x}}_{n,n} + \mathbf{G} \mathbf{u}_n
    predicted_state_mean = A @ current_state_mean
    
    # Predict the next covariance: \mathbf{P}_{n+1,n} = \mathbf{F} \mathbf{P}_{n,n} \mathbf{F}^T + \mathbf{Q}
    predicted_state_covariance = A @ current_state_covariance @ A.T + W
    
    # Save the prediction
    predicted_state_means.append(predicted_state_mean[0])
    
    # Get the current observation
    current_observation = test_windspeed[t].reshape(1, 1)
    
    # Measurement Update (Correct)
    # Compute the Kalman Gain: \mathbf{K}_n = \mathbf{P}_{n,n-1} \mathbf{H}^T (\mathbf{H} \mathbf{P}_{n,n-1} \mathbf{H}^T + \mathbf{R}_n)^{-1}
    S = H @ predicted_state_covariance @ H.T + Q
    K = predicted_state_covariance @ H.T @ np.linalg.inv(S)
    
    # Update the state estimate with measurement: \hat{\mathbf{x}}_{n,n} = \hat{\mathbf{x}}_{n,n-1} + \mathbf{K}_n (\mathbf{z}_n - \mathbf{H} \hat{\mathbf{x}}_{n,n-1})
    current_state_mean = predicted_state_mean + K @ (current_observation - H @ predicted_state_mean)
    
    # Update the estimate uncertainty: \mathbf{P}_{n,n} = (\mathbf{I} - \mathbf{K}_n \mathbf{H}) \mathbf{P}_{n,n-1}
    current_state_covariance = (np.eye(1) - K @ H) @ predicted_state_covariance
    
    # Save the filtered state
    filtered_state_means.append(current_observation[0])

# Calculate evaluation metrics
mae = mean_absolute_error(filtered_state_means, predicted_state_means)
mse = mean_squared_error(filtered_state_means, predicted_state_means)
rmse = np.sqrt(mse)
r2 = r2_score(filtered_state_means, predicted_state_means)

print(f"MAE: {mae:.4f}")
print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R-squared: {r2:.4f}")

# Plot results
pl.figure(figsize=(16, 6))
pl.plot(range(train_size, train_size + len(test_windspeed)), filtered_state_means, linestyle='-', color='b', label='True Wind On/Offshore (wind_on_offshore)')
pl.plot(range(train_size, train_size + len(test_windspeed)), predicted_state_means, linestyle='--', color='g', label='Predicted Wind On/Offshore')
pl.legend()
pl.xlabel('Time')
pl.ylabel('Normalized Value')
pl.show()


# Train and test data

In [ ]:
df_train = df_daily.loc[df_daily["timestamps"]<"2021-12-31 23:00:00"]
df_test = df_daily.loc[df_daily["timestamps"]>="2021-12-31 23:00:00"]

In [ ]:
from pykalman import KalmanFilter
import numpy as np
from sklearn.metrics import mean_squared_error

kf = KalmanFilter(n_dim_obs=1,n_dim_state=2,em_vars=['transition_matrices', 'observation_matrices', 'transition_covariance', 'observation_covariance', 'initial_state_mean', 'initial_state_covariance']
                  )
predictions = []
actual = []

train_data = df_daily.loc[df_daily["timestamps"]<"2021-12-31 23:00:00"]
predict_data = df_daily.loc[df_daily["timestamps"]>"2021-12-31 23:00:00"]
kf.em(train_data["photo"], n_iter=300)
for t in range(len(predict_data)):
    oberservation = predict_data["photo"].iloc[t]
    if t%100==0:
        print(f"the obersevation is: {oberservation} at time {t}")
        print(f"Transition Matrix (F):\n{kf.transition_matrices} at time {t}")
        filtered_state_means, filtered_state_covariances = kf.filter(oberservation)
        forecasted_state_mean = np.dot(kf.transition_matrices, filtered_state_means[-1])
        forecasted_value = forecasted_state_mean[0]
        print(f"Forecasted value: {forecasted_value} at time {t}")
        print(f"Filtered state mean: {filtered_state_means[-1]} at time {t}")
        print(f"Filtered state covariances: {filtered_state_covariances[-1]} at time {t}")
        print(f"Init state mean{kf.initial_state_mean} at time {t}")
        
        
    prediction_state_mean, _ = kf.filter_update(kf.initial_state_mean, kf.initial_state_covariance, oberservation)
    kf.filter
    predictions.append(prediction_state_mean[0])
    if t%100==0:
        print(f"This is the prediction: {prediction_state_mean[0]} at time {t}")
        print("-------------------------\n")
    actual.append(oberservation)
    
    
    
rmse = np.sqrt(mean_squared_error(actual, predictions))

print(f"RMSE: {rmse}")

notmup-3zigci-bisFeg

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(predict_data.index, actual, label='Actual')
plt.plot(predict_data.index, predictions, label='Predicted')
plt.xlabel('Timestamp')
plt.ylabel('Wind Onshore')
plt.title('Actual vs Predicted Wind Onshore (2022)')
plt.legend()
plt.show()

In [ ]:
from pykalman import KalmanFilter
import numpy as np
from sklearn.metrics import mean_squared_error

kf = KalmanFilter(n_dim_obs=1,n_dim_state=5,em_vars=['transition_matrices', 'observation_matrices', 'transition_covariance', 'observation_covariance', 'initial_state_mean', 'initial_state_covariance']
                  )
predictions = []
actual = []

train_data = df_daily.loc[df_daily["timestamps"]<"2021-12-31 23:00:00"]
predict_data = df_daily.loc[df_daily["timestamps"]>"2021-12-31 23:00:00"]
kf.em(train_data["photo"], n_iter=300)
for t in range(len(predict_data)):
    observation = predict_data['photo'].iloc[t]
    
    # Use filter() to get the filtered state estimates up to current time t
    filtered_state_means, filtered_state_covariances = kf.filter(observation)
    
    # Use filter_update() to update state estimate with current observation
    prediction_state_mean, _ = kf.filter_update(filtered_state_means[-1], filtered_state_covariances[-1], observation)
    
    # Store prediction and actual value for comparison
    predictions.append(prediction_state_mean[0])
    actual.append(observation)
    
    # print information
    if t%100==0:
        print(f"the obersevation is: {observation} at time {t}")
        print(f"Transition Matrix (F):\n{kf.transition_matrices} at time {t}")
        print(f"Forecasted value: {prediction_state_mean[0]} at time {t}")
        print(f"Filtered state mean: {filtered_state_means[-1]} at time {t}")
        print(f"Filtered state covariances: {filtered_state_covariances[-1]} at time {t}")
        print(f"Init state mean{kf.initial_state_mean} at time {t}")
        print("-------------------------\n")
        

    
    
    
    
rmse = np.sqrt(mean_squared_error(actual, predictions))

print(f"RMSE: {rmse}")

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(predict_data.index, actual, label='Actual')
plt.plot(predict_data.index, predictions, label='Predicted')
plt.xlabel('Timestamp')
plt.ylabel('Wind Onshore')
plt.title('Actual vs Predicted Wind Onshore (2022)')
plt.legend()
plt.show()

In [ ]:
df_train

In [ ]:
import numpy as np
import pylab as pl
from pykalman import KalmanFilter

# Example data (replace with your actual data)
np.random.seed(42)  # For reproducibility
n_timesteps = 100
solar_radiation = np.random.rand(n_timesteps)  # Simulated solar radiation data
pv_output = 2 * solar_radiation + np.random.randn(n_timesteps) * 0.1  # Simulated photovoltaic output data

# Normalize the data
solar_radiation = (solar_radiation - np.mean(solar_radiation)) / np.std(solar_radiation)
pv_output = (pv_output - np.mean(pv_output)) / np.std(pv_output)

# Prepare matrices
x = df
z = solar_radiation.reshape(-1, 1)  # Observations (solar radiation)

M = n_timesteps

# Estimate A, W, H, Q using the closed-form solutions
A = np.dot(x[1:].T, x[:-1]) @ np.linalg.inv(np.dot(x[:-1].T, x[:-1]))
W = (np.dot(x[1:].T, x[1:]) - np.dot(A, np.dot(x[:-1].T, x[1:]))) / (M - 1)
H = np.dot(z.T, x) @ np.linalg.inv(np.dot(x.T, x))
Q = (np.dot(z.T, z) - np.dot(H, np.dot(x.T, z))) / M

# Convert to correct shapes
A = A.reshape(1, 1)
W = W.reshape(1, 1)
H = H.reshape(1, 1)
Q = Q.reshape(1, 1)

# Initialize Kalman Filter with estimated parameters
kf = KalmanFilter(
    transition_matrices=A,
    observation_matrices=H,
    transition_covariance=W,
    observation_covariance=Q,
    initial_state_mean=x[0],
    initial_state_covariance=np.eye(1)
)

# Filter the observations to estimate the states
filtered_state_estimates = kf.filter(z)[0]

# Plot results
pl.figure(figsize=(16, 6))
lines_true = pl.plot(x, linestyle='-', color='b', label='True PV Output (hidden state)')
lines_obs = pl.plot(z, linestyle=':', color='m', label='Solar Radiation (observation)')
lines_filt = pl.plot(filtered_state_estimates, linestyle='--', color='g', label='Filtered PV Output (state estimate)')
pl.legend()
pl.xlabel('Time')
pl.ylabel('Normalized Value')
pl.show()


In [ ]:
data

In [ ]:
data.observations

In [ ]:
data.states

In [ ]:
import numpy as np
import pylab as pl
from pykalman.datasets import load_robot
from pykalman import KalmanFilter

# Load data (we'll only use data.observations for this example)
data = load_robot()

# Define dimensions
n_dim_state = data.transition_matrix.shape[0]  # State dimension
n_dim_obs = data.observation_matrix.shape[0]   # Observation dimension

# Initialize with random values
np.random.seed(42)  # For reproducibility
transition_matrix = np.random.randn(n_dim_state, n_dim_state)
observation_matrix = np.random.randn(n_dim_obs, n_dim_state)
transition_covariance = np.eye(n_dim_state)  # Identity matrix for simplicity
observation_covariance = np.eye(n_dim_obs)   # Identity matrix for simplicity
transition_offsets = np.random.randn(n_dim_state)
observation_offset = np.random.randn(n_dim_obs)
initial_state_mean = np.random.randn(n_dim_state)
initial_state_covariance = np.eye(n_dim_state)  # Identity matrix for simplicity

# Initialize Kalman Filter
kf = KalmanFilter(
    transition_matrices=transition_matrix,
    observation_matrices=observation_matrix,
    transition_covariance=transition_covariance,
    observation_covariance=observation_covariance,
    transition_offsets=transition_offsets,
    observation_offsets=observation_offset,
    initial_state_mean=initial_state_mean,
    initial_state_covariance=initial_state_covariance,
    em_vars=[
        'transition_matrices', 'observation_matrices',
        'transition_covariance', 'observation_covariance',
        'observation_offsets', 'initial_state_mean',
        'initial_state_covariance'
    ]
)

# Learn good values for parameters named in `em_vars` using the EM algorithm
loglikelihoods = np.zeros(10)
for i in range(len(loglikelihoods)):
    kf = kf.em(X=data.observations, n_iter=1)
    loglikelihoods[i] = kf.loglikelihood(data.observations)

# Estimate the state without using any observations (blind estimation)
n_timesteps = data.observations.shape[0]
blind_state_estimates = np.zeros((n_timesteps, n_dim_state))
for t in range(n_timesteps - 1):
    if t == 0:
        blind_state_estimates[t] = kf.initial_state_mean
    blind_state_estimates[t + 1] = (
        np.dot(kf.transition_matrices, blind_state_estimates[t])
        + kf.transition_offsets
    )

# Estimate the hidden states using observations up to and including time t
filtered_state_estimates = kf.filter(data.observations)[0]

# Estimate the hidden states using all observations
smoothed_state_estimates = kf.smooth(data.observations)[0]

# Plot the true, blind, filtered, and smoothed state estimates
pl.figure(figsize=(16, 6))
lines_true = pl.plot(data.states, linestyle='-', color='b')
lines_blind = pl.plot(blind_state_estimates, linestyle=':', color='m')
# lines_filt = pl.plot(filtered_state_estimates, linestyle='--', color='g')
lines_smooth = pl.plot(smoothed_state_estimates, linestyle='-.', color='r')
pl.legend(
    (lines_true[0], lines_blind[0], lines_filt[0], lines_smooth[0]),
    ('true', 'blind', 'filtered', 'smoothed')
)
pl.xlabel('time')
pl.ylabel('state')
pl.xlim(xmax=500)

# Draw log likelihood of observations as a function of EM iteration number.
# Notice how it is increasing (this is guaranteed by the EM algorithm)
pl.figure()
pl.plot(loglikelihoods)
pl.xlabel('em iteration number')
pl.ylabel('log likelihood')
pl.show()


In [ ]:
data.observation_matrix

In [ ]:
import numpy as np
import pylab as pl
from pykalman import KalmanFilter

# Example data (replace with your actual data)
np.random.seed(42)  # For reproducibility
n_timesteps = 100
solar_radiation = np.random.rand(n_timesteps)  # Simulated solar radiation data
pv_output = 2 * np.random.rand(n_timesteps) + np.random.randn(n_timesteps) * 0.1  # Simulated photovoltaic output data

# Dimensions
n_dim_state = 1  # We assume the state is one-dimensional (PV output)
n_dim_obs = 1  # We assume the observation is one-dimensional (solar radiation)

# Random initialization
transition_matrix = np.eye(n_dim_state)  # F: Identity matrix for state transition
observation_matrix = np.random.randn(n_dim_obs, n_dim_state)  # H: Random initialization
initial_state_mean = np.random.randn(n_dim_state)  # Initial state
initial_state_covariance = np.eye(n_dim_state)  # Initial state covariance
transition_covariance = np.eye(n_dim_state)  # Q: Process noise covariance
observation_covariance = np.eye(n_dim_obs)  # R: Measurement noise covariance
transition_offsets = np.zeros(n_dim_state)  # Transition offsets
observation_offset = np.zeros(n_dim_obs)  # Observation offsets

# Initialize Kalman Filter with random parameters
kf = KalmanFilter(
    transition_matrices=transition_matrix,
    observation_matrices=observation_matrix,
    transition_covariance=transition_covariance,
    observation_covariance=observation_covariance,
    transition_offsets=transition_offsets,
    observation_offsets=observation_offset,
    initial_state_mean=initial_state_mean,
    initial_state_covariance=initial_state_covariance,
    em_vars=[
        'transition_matrices', 'observation_matrices',
        'transition_covariance', 'observation_covariance',
        'observation_offsets', 'initial_state_mean',
        'initial_state_covariance'
    ]
)

# Observations are the solar radiation data
observations = solar_radiation.reshape(-1, 1)

# Estimate parameters using EM algorithm
loglikelihoods = np.zeros(140)
for i in range(len(loglikelihoods)):
    kf = kf.em(X=observations, n_iter=1)
    loglikelihoods[i] = kf.loglikelihood(observations)

# Filtering
filtered_state_estimates = kf.filter(observations)[0]
print(f"Transtition matrix: {kf.transition_matrices}")
print(f"Observatuib matrix: {kf.observation_matrices}")

# Plot results
pl.figure(figsize=(16, 6))
line_true = pl.plot(pv_output, linestyle='-', color='r', label='True values')
lines_obs = pl.plot(observations, linestyle='-', color='b', label='Solar Radiation (observations)')
lines_filt = pl.plot(filtered_state_estimates, linestyle='--', color='g', label='Filtered PV Output (state estimate)')
pl.legend()
pl.xlabel('Time')
pl.ylabel('Value')
pl.show()

# Plot log likelihoods
pl.figure()
pl.plot(loglikelihoods)
pl.xlabel('EM iteration number')
pl.ylabel('Log likelihood')
pl.show()


In [ ]:
import pandas as pd
from darts import TimeSeries
from darts.models import KalmanForecaster
from darts.metrics import mape
from darts.metrics.metrics import rmse, mae, r2_score

In [ ]:
df_train = df_hourly.loc[df_hourly["timestamps"]<="2021-12-31 23:00:00"]
df_test = df_hourly.loc[df_hourly["timestamps"]>"2021-12-31 23:00:00"]
df_test.fillna(0, inplace=True)
df_train.fillna(0, inplace=True)

In [ ]:
df_test

In [ ]:



series = TimeSeries.from_dataframe(df_train, "timestamps", ["photo"])
series_test = TimeSeries.from_dataframe(df_test, "timestamps", ["photo"])

model = KalmanForecaster(dim_x=60)
model.fit(series)




In [ ]:
forecast = model.predict(10)
# Validation




plt.figure(figsize=(10, 6))
series.plot(label='train')
series_test.plot(label='test_vals')
forecast.plot(label='Forecast')

plt.legend()
plt.title('Kalman Filter Forecast using N4SID')
plt.show()